In [1]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json 
import requests
import datetime
from datetime import date
import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 800)

In [2]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key='a539d7df2c7b43e1ac4d12f386d901e8')

In [3]:
x = newsapi.get_top_headlines(country='us', page_size=50, category = 'health')

In [4]:
nyt_api_key = '13bd501bc77542a58e2e6678619b0d60'

In [5]:
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

# NYT & NewsAPI

In [18]:
import requests
import re
import numpy as np
import time
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
import datetime
from datetime import date
from apikeys import *
from info import *

#dates to use for API call
today = int(str(date.today()).replace('-',''))
last_week = int(str(date.today() - datetime.timedelta(days = 14)).replace('-',''))

#NEW YORK TIMES
#clean the response from NYT API
def NYT_title_clean(df):
    titles = []
    for index, row in df.iterrows():
        title = row.headline['main']
        titles.append(title)
    df['title'] = titles
    return df

def NYT_dropped_rows(df):
    df.pub_date = pd.to_datetime(df.pub_date).dt.date
    df.word_count = round(df.word_count / 150)
    df.document_type = 'text'
    df['formality'] = 'Intermediate'
    return df

def NYT_dataframe_clean(df):
    dataframe = NYT_title_clean(df)
    dataframe = NYT_dropped_rows(dataframe)
    return dataframe

def NYT_api_call_section_based(section, source, page, start, end, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:({section_name})&page={page}&source:({source})&begin_date={start}&end_date={end}&api-key={api}'.format(section_name = section, page = page, source = source, start = start, end = end, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = section
    df['image_url'] = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHEtiVXw8Wi1tp56Nzd5rH_EoOAJA2RInEWvf5h5CQ-6O_YZp7dw'
    return df

def NYT_api_call_parameter_ALLTIME(param, page, key):
    url = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?q={param}&page={page}&sort=newest&&api-key={api}'.format(param = param, page = page, api = key)
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    df = pd.DataFrame(data['response']['docs'])
    df = NYT_dataframe_clean(df)
    df['param'] = param
    df['image_url'] = 'https://greaterbostonhcs.com/wp-content/uploads/2016/05/Nutrition.jpg'
    return df

def NYT_pull(categories):
    empty = pd.DataFrame()
    for word in categories:
        try:
            df = NYT_api_call_parameter_ALLTIME(word,0,nyt_api_key)
            empty = empty.append(df, sort=True)
            print('Pulled '+word)
            time.sleep(2)
        except:
            print(word + " EXCEPTION!!!!")
    empty = empty.drop(['abstract','section_name'],axis = 1)
    empty = empty.rename(index=str, columns={"_id": "source_id", "document_type": "medium",'pub_date':'date','snippet':'description','word_count':'length'})
    return empty

#NEWSAPI

def rename_columns(df):
    df = df.rename(index=str, columns={'publishedAt':'date','url':'web_url','urlToImage':'image_url'})
    return df

def add_words(df):
    lengths = []
    for string in df.content:
        try:
            lengths.append(round(int(string[string.find('+')+1:string.find(' chars')]) / 4 / 250))
        except:
            lengths.append(4)
    return lengths

def split_source_info(list_of_dicts):
    for item in list_of_dicts:
        item['source_id'] = item['source']['id']
        item['source'] = item['source']['name']

def pull_articles(parameter):
    article_results_rel = newsapi.get_everything(q=parameter,sort_by = 'relevancy',language='en', page_size=10, sources=sources_joined)
    article_results_rel = article_results_rel['articles']
    split_source_info(article_results_rel)
    return article_results_rel

def clean_articles(list_of_dicts, search_param):
    df = pd.DataFrame(list_of_dicts)
    try:
        df['medium'] = 'text'
        df['param'] = search_param
        df['publishedAt'] = df['publishedAt'].apply(lambda x: pd.to_datetime(x).date().strftime('%Y-%m-%d'))
        df['formality'] = 'Intermediate'
        df['length'] = add_words(df)
        df = rename_columns(df)
        print(search_param)
    except:
        pass
    return df


def call_news_api(categories):
    empty_df = pd.DataFrame()
    for category in categories:
        dicts = pull_articles(category)
        df = clean_articles(dicts, category)
        try:
            empty_df = empty_df.append(df, sort=True)
        except:
            pass
        print(len(empty_df.index))
    return empty_df


# Social Media

In [214]:
import tweepy
from apikeys import *
import pandas as pd
import json
import requests
import datetime
from datetime import date
from info import *

auth = tweepy.OAuthHandler(twitter_1, twitter_2)
auth.set_access_token(twitter_3, twitter_4)
api = tweepy.API(auth)
#all twitter handles to scrape
# twitter_handles = ['@ATPScience1', '@waitrose', '@MicrobiomeInst', '@veganrecipescom', '@cldiet', '@Onnit', '@vegsoc', '@VeganKosher', '@TheVeganSociety', '@vegan', '@Keto_Recipes_', '@the52diet', '@IFdiet', '@microbiome', '@metagenomics', '@microbiome_news', '@TheGutStuff', '@MyGutHealth', '@PaleoFX',
# '@PaleoFoundation', '@ThePaleoDiet', '@PaleoComfort', '@cavemanketo', '@KetoFlu', '@TheKetoKitchen_', '@EatKetoWithMe', '@KetoConnect', '@KetoDietZone', '@Ketogenic', '@USDANutrition', '@FoodRev', '@CSPI', '@simplyrecipes', '@FoodNetwork', '@CookingChannel', '@tasty', '@nytfood', '@finecooking', '@mrcookingpanda'
# , '@FODMAPeveryday', '@FODMAPLife', '@FodmappedInfo', '@thefodmapdoctor', '@SimplyGlutenFre', '@gfliving', '@sibotest', '@manjulaskitchen', '@VegTimes', '@CookingLight', '@mealprepwl', '@thehealthygut', '@VitalGutHealth', '@pureguthealth', '@PaleoForBegin', '@PaleoLeap', '@ThePaleoMom', '@paleomagazine', '@PaleoHacks', '@paleogrubs',
# '@naturalgourmet', '@Low_Carb_Keto', '@NutritionTwins', '@mckelhill', '@WomensFitnessAu', '@WomensHealthMag', '@MensHealthMag', '@mjfit', '@thugkitchen', '@Leslie_Klenke', '@insidePN', '@ThisMamaCooks', '@EdibleWildFood', '@TheEarthDieter', '@HarvardHealth', '@EverydayHealth', '@DailyHealthTips']

#clean response from twitter

def clean_tweets(data, categories):
    topics = []
    for tweet in data:
        try:
            hashtag = tweet.entities['hashtags'][0]['text']
            tags = list(pd.DataFrame(tweet.entities['hashtags']).text)
            intersect = list(set(tags).intersection(categories))
            if len(intersect) > 0:
                hashtag = intersect[0]
            else:
                hashtag = hashtag
        except IndexError:
            words = set(re.sub("[^\w]", " ",  tweet.text).split())
            int2 = list(words.intersection(categories))
            if len(int2) > 0:
                hashtag = int2[0]
            else:
                hashtag = 'general'
        topics.append(hashtag)
    tweets = [{'title':tweet.id, 'date':tweet.created_at.date().strftime('%Y-%m-%d'),
       'description': tweet.text, 'source':tweet.user.screen_name,'source_id':tweet.user.id_str,
       'formality': 'Informal'
       ,'length': 1,'medium':'text'} for tweet in data]
    tweets = pd.DataFrame(tweets)
    tweets['param'] = topics
    return tweets



#CALL API
def twitter_api_call(list_handles, categories):
    empty = pd.DataFrame()
    for handle in list_handles:
        user_tweets = pd.DataFrame(clean_tweets(api.user_timeline(handle), categories))
        empty = empty.append(user_tweets, sort=True)
        print(handle)
    empty.title = empty.title.astype('str')
    empty['web_url'] = 'https://twitter.com/'+empty.source+'/status/'+empty.title
    empty['image_url'] = empty['web_url']
    return empty
import requests

class Tweet(object):
    def __init__(self, s, embed_str=False):
        if not embed_str:
            # Use Twitter's oEmbed API
            # https://dev.twitter.com/web/embedded-tweets
            api = 'https://publish.twitter.com/oembed?url={}'.format(s)
            response = requests.get(api)
            self.text = response.json()["html"]
        else:
            self.text = s

    def _repr_html_(self):
        return self.text


# Youtube 

In [103]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import pprint
import pafy
import warnings
warnings.filterwarnings("ignore")
from apikeys import *
from info import *


DEVELOPER_KEY = youtube_key
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"


def clean_youtube_time(string):
    if 'H' in string:
        minutes = int(string[string.find('H')+1:string.find('M')])
        hours = int(string[string.find('T')+1:string.find('H')]) * 60
        time = minutes + hours
    else:
        if 'M' in string:
            time = int(string[string.find('T')+1:string.find('M')])
        else:
            time = 1
    return time

def youtube_search(q, max_results=10,order="date", token=None, location=None, location_radius=None):

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

    search_response = youtube.search().list(
    q=q,
    type="video",
    pageToken=token,
    order = order,
    part="id,snippet", # Part signifies the different types of data you want
    maxResults=max_results,
    location=location,
    locationRadius=location_radius).execute()

    all_dicts = []

    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":

            title = (search_result['snippet']['title'])

            videoId = (search_result['id']['videoId'])

            response = youtube.videos().list(
            part='statistics, snippet, contentDetails',
            id=search_result['id']['videoId']).execute()

            channelId = (response['items'][0]['snippet']['channelId'])
            channelTitle = (response['items'][0]['snippet']['channelTitle'])
            categoryId = (response['items'][0]['snippet']['categoryId'])
            favoriteCount = (response['items'][0]['statistics']['favoriteCount'])
            viewCount = (response['items'][0]['statistics']['viewCount'])
            date = pd.to_datetime((response['items'][0]['snippet']['publishedAt'])).date().strftime('%Y-%m-%d')
            description = response['items'][0]['snippet']['localized']['description']
            url = 'https://www.youtube.com/watch?v='+videoId
            image_url = response['items'][0]['snippet']['thumbnails']['default']['url']
            length = clean_youtube_time(response['items'][0]['contentDetails']['duration'])

        if 'commentCount' in response['items'][0]['statistics'].keys():
            commentCount = (response['items'][0]['statistics']['commentCount'])
        else:
            commentCount = []

        if 'tags' in response['items'][0]['snippet'].keys():
            tags = (response['items'][0]['snippet']['tags'])
        else:
            tags = []

        youtube_dict = {'tags':tags,'source_id': channelId,'source': channelTitle,'categoryId':categoryId,'title':title,'videoId':videoId,'viewCount':viewCount,'commentCount':commentCount,'favoriteCount':favoriteCount,
                        'formality':'Intermediate', 'medium':'video','date':date, 'description': description, 'web_url':url, 'image_url':image_url, 'length':length}
        all_dicts.append(youtube_dict)
    return pd.DataFrame(all_dicts)

def add_category(df, categories):
    # cats = ['keto','ketogenic','paleo','paleolithic','vegan','vegetarian']
    all_params = []
    for index, row in df.iterrows():
        try:
            intersect = list(set(row.tags).intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = row.tags[0]
        except:
            category = 'none'
        all_params.append(category)
    df['param'] = all_params
    return df

def youtube_api_call(list_accounts, categories):
    empty_df = pd.DataFrame()
    errors = []
    for account in list_accounts:
        try:
            df = youtube_search(account)
            df = add_category(df, categories)
            empty_df = empty_df.append(df, sort=True)
            print(account)
        except:
            print(account + " EXCEPTION!!!!")
    return empty_df

# FB AND INSTA

In [10]:
# from instagram.client import InstagramAPI
# api = InstagramAPI(client_id="EAACl5okwUhQBAD6vhJiELgsatruedytMV67mvDuN2wgXEyRAXG7umE3T0KEweMhlWQWPgku5pF8KBwwOy9JJuFxx2chbYxPTjMdYA2mTd1DL6Jd5t4XwcetZBwhZC1Pyrl5MW2X90w6J9ZCZCWYlODApHzosU7ReUPZBmFITJj0cpsMZCDZATmEKip23ZA1nyA40Dv8IsLccLAZDZD")
# popular_media = api.media_popular(count=20)
# for media in popular_media:
#     print(media.images['standard_resolution'].url)

In [11]:
import facebook

graph = facebook.GraphAPI(access_token="EAACl5okwUhQBANPTECp6sKqxm3vubZCFmBBEtwdUQgLPAvAFLhpmk6ZAgDTB6Q6g6y6lvz3yjf7iZBW2ELCSrAwDhp28A1xse1QkQrvXowYgivMP0Rz3NRO8cEEy514LC7x3pAmFr1RBZBe7RJVVlsY70OWLHOgceSZCZC0HqzFdw13oEkEZC0Bm4ehPBxTlJFOoyiFE5f8ZCAZDZD", version="2.12")

# AUDIO

In [299]:
import requests
import json
import feedparser
import pandas as pd
import re
import time


def feed_urls(search_words, media_value='podcast', entity_value='podcast'):
#     Args:
#         search_words: The URL-encoded text string to be searched for
#         media_value: {movie, podcast, music, musicVideo, audiobook,
#                     shortFilm, tvShow, software, ebook, all} optional
#                     An optional variable, which indicates the media type to be searched for.
#         entity_value: Optional

    payload = {'term': search_words, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        feed_url = itunes_result_json["results"][0]['feedUrl']
    else:
        feed_url = "None"
    return feed_url

def fix_podcast_length(time):
    hours = int(time[0:2]) * 60
    minutes = int(time[4:5])
    length = hours + minutes
    return length

def df_podcast_episodes(feed_url):
    print(feed_url)
    if(len(feed_url) > 0):
        feed = feedparser.parse(feed_url)
        episodes = []
        for episode in feed.entries:
            info = dict()
            info['title'] = episode['title'] if 'title' in episode else ''
            info['description']= episode['summary'] if 'summary' in episode else ''
            info['length']= fix_podcast_length(episode['itunes_duration']) if 'itunes_duration' in episode else 1
            info['date']= pd.to_datetime(episode['published']).date().strftime('%Y-%m-%d') if 'published' in episode else ''
            info['medium'] = 'audio'
            info['formality'] = 'Intermediate'
            info['source'] = feed['feed']['title'] if 'title' in feed['feed'] else ''
            info['source_id'] = feed['feed']['title_detail']['base'] if 'title_detail' in feed['feed'] else ''
            try:
                info['web_url'] = episode['links'][0]['href'] if 'links' in episode else ''
            except:
                info['web_url'] = feed['feed']['title_detail']['base'] if 'title_detail' in feed['feed'] else ''
            info['image_url'] = feed['feed']['image']['href'] if 'image' in feed['feed'] else ''
            episodes.append(info)
        df = pd.DataFrame(episodes)
        return df
    else:
        print("No response!")

def add_category_to_audio(df, categories):
    all_params = []
    for index, row in df.iterrows():
        try:
            words = set(re.sub("[^\w]", " ",  row.description).split())
            intersect = list(words.intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = 'general'
        except:
            category = 'general'
        all_params.append(category)
    df['param'] = all_params
    return df

def call_podcast_api(categories, podcasts):
    empty = pd.DataFrame()
    for podcast in podcasts:
        if podcast == 'None':
            pass
        else:
            try:
                url = feed_urls(podcast)
                df = df_podcast_episodes(url)
                df = add_category_to_audio(df, categories)
                empty = empty.append(df, sort=True)
                print('SUCCESS!')
                time.sleep(2)
            except:
                print('EXCEPTION')
                time.sleep(2)
    return empty


# EBOOKS

In [334]:
pd.set_option('display.max_columns', None)

In [368]:
def clean_ebook_date(df):
    dates = []
    for index, row in df.iterrows():
        date = pd.to_datetime(row.date).date().strftime('%Y-%m-%d')
        dates.append(date)
    df['date'] = dates
    return df


def ebook_search(search_word, media_value='ebook', entity_value='ebook'):
    payload = {'term': search_word, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        df = pd.DataFrame(itunes_result_json['results'])
        #NOTE LENGTH IS ACTUALLY THE PRICE BUT USE SAME LABEL FOR CONSISTENCY
        df = df.rename(index = str, columns= {'artistName': 'source','trackViewUrl':'web_url',
                                        'artworkUrl100':'image_url','price': 'length','releaseDate':'date','trackName':'title'})
        df['source_id'] = 'Itunes Ebook'
        df['formality'] = 'Formal'
        df['medium'] = 'text'
        df['param'] = search_word
        df = df.fillna(1)
        return clean_ebook_date(df)
    else:
        print('No Results!')
        return 'Empty'

def call_ebook_api(categories):
    empty = pd.DataFrame()
    for category in categories:
        df = ebook_search(category)
        if type(df) != str:
            empty = empty.append(df, sort=True)
            time.sleep(2)
            print('Added '+category)
        else:
            pass
    return empty


# Movies/Docs

In [472]:
def add_category_to_movie(df, categories):
    all_params = []
    for index, row in df.iterrows():
        try:
            words = set(re.sub("[^\w]", " ",  row.description).split())
            intersect = list(words.intersection(categories))
            if len(intersect) > 0:
                category = intersect[0]
            else:
                category = 'general'
        except:
            category = 'general'
        all_params.append(category)
    df['param'] = all_params
    return df

def movie_search(search_word,categories, media_value='movie', entity_value='movie'):
    payload = {'term': search_word, 'media': media_value, 'entity' : entity_value}
    itunes_request = requests.get('https://itunes.apple.com/search', params=payload)
    itunes_result_json = itunes_request.json()
    result_count = itunes_result_json["resultCount"]
    if result_count > 0:
        df = pd.DataFrame(itunes_result_json['results'])
        df = df.rename(index = str, columns= {'artistName': 'source','trackViewUrl':'web_url',
                                        'artworkUrl100':'image_url','trackTimeMillis': 'length'
                                              ,'releaseDate':'date','trackName':'title', 'longDescription':'description'})
        df['source_id'] = 'Itunes Movie'
        df['formality'] = 'Formal'
        df['medium'] = 'video'
        df['length'] = round(df['length'] / 60000)
        df = df.fillna(1)
        df = clean_ebook_date(df)
        return add_category_to_movie(df,categories)
    else:
        print('No Results!')
        return 'Empty'

def call_movie_api(movies, categories):
    empty = pd.DataFrame()
    for movie in movies:
        try:
            df = movie_search(movie, categories)
            if type(df) != str:
                empty = empty.append(df, sort=True)
                time.sleep(5)
                print('Added '+movie)
            else:
                time.sleep(5)
        except:
            print(movie + ' EXCEPTION')
    return empty


In [463]:
movie_list = ['Somebody Feed Phil' , 'Meat Eater' , 'Salt, Fat, Heat, Acid' , 'For Grace' , 'Chef vs Science: The Ultimate Kitchen Challenge' , 'Ugly Delicious' , 'Avec Eric' , 'The Great British Baking Show' , 'Lords and Ladles' , 'Cooking on High' , 'The Mind of a Chef' , 'Anthony Bourdain Parts Unknown' , 'Jeremiah Tower: The Last Magnificent' , 'In Search of Israeli Cuisine' , 'Theater of Life' , 'Sour Grapes' , 'Chefs Table' , 'Noma: My Perfect Storm' , 'The Birth of Saké' , 'Barbecue' , 'A Year in Champagne' , '42 grams' , 'Sugar Coated' , 'Somm' , 'More Than Honey' , 'Jiro Dreams of Sushi' , 'Spinning Plates' , 'The Future of Food' , 'Ingredients' , 'Simply Raw: Reversing Diabetes in 30 Days' , 'Sustainable' , 'A Place at the Table' , 'Farmageddon' , 'Bite Size' , 'Food Chains' , 'Plant Pure Nation' , 'Super Size Me' , 'Food Matters' , 'Food Choices' , 'In Defense of Food' , 'What the Health' , 'GMO OMG' , 'Cowspiracy' , 'Vegucated' , 'Fat, Sick & Nearly Dead' , 'Fed Up' , 'Food Inc.' , 'Hungry for Change' , 'Rotten' , 'Cooked' , 'Forks Over Knives' , 'The Magic Pill']

In [464]:
nutrition_cats = ['keto',  'mct', 'natural', 'corn-fed', 'cruciferous','paleolithic',
'vegetable', 'cooking', 'factory', 'fat', 'anti-biotic', 'grass-fed', 'gluten', 'vegan', 'vegetarian'
,'paleo', 'ketogenic', 'leaky gut', 'meal prep', 'micronutrient', 'macronutrient', 'pressure cooked carbs', 'sprouted carbs', 'mct oil', 'coconut oil', 'lean protein', 'fat on nutrion label', 'cholestrol', 'natural sugars', 'diabetes', 'brain fog', 'sourdough', 'bcaas', 'essential amino acids'
, 'wild fish', 'farm raised fish', 'corn-fed beef', 'wild game', 'bioavailability', 'amino acids', 'meat', 'grains', 'vitamins', 'supplements', 'sprouts', 'beans', 'legumes', 'lectins', 'night shade', 'auto-immune', 'hormone', 'gluconeogenesis', 'bad fat', 'good fat'
, 'root veggies', 'cruciferous vegetables', 'dark leafy greens', 'anti-inflammatory', 'infloamation', 'energy', 'mood', 'gut brain connection', 'wheat belly', 'fermented', 'vegetable oil', 'cooking oils', 'glysophate', 'pesticides', 'factory farming', 'sugar', 'fat loss', 'free range', 'anti-biotic free', 'organic'
, 'grass-fed dairy', 'grass-fed protein', 'plant based protein', 'animal protein', 'protein', 'blood glucose', 'fiber', 'starch', 'refined sugar', 'refined carbs', 'simple carbs', 'complex carbs', 'carbs', 'white hdl fat', 'brown ldl fat', 'fats', 'prebiotic', 'enzymes', 'probiotic', 'microbiome'
, 'foodmap diet']

In [459]:
test_cats = ['sushi', 'keto']
test_movies = ['Jiro Dreams of Sushi', 'The Magic Pill', 'Cooked']

In [473]:
test = call_movie_api(movie_list,nutrition_cats)

No Results!
No Results!
No Results!
Added For Grace
No Results!
No Results!
Added Avec Eric
No Results!
No Results!
No Results!
No Results!
No Results!
Added Jeremiah Tower: The Last Magnificent
Added In Search of Israeli Cuisine
No Results!
Added Sour Grapes
No Results!
Added Noma: My Perfect Storm
Added The Birth of Saké
Added Barbecue
Added A Year in Champagne
Added 42 grams
No Results!
Added Somm
Added More Than Honey
Added Jiro Dreams of Sushi
Added Spinning Plates
Added The Future of Food
Added Ingredients
No Results!
Added Sustainable
Added A Place at the Table
Added Farmageddon
Bite Size EXCEPTION
Added Food Chains
Plant Pure Nation EXCEPTION
Added Super Size Me
Food Matters EXCEPTION
Food Choices EXCEPTION
Added In Defense of Food
Added What the Health
GMO OMG EXCEPTION
No Results!
Added Vegucated
Fat, Sick & Nearly Dead EXCEPTION
Fed Up EXCEPTION
Added Food Inc.
Added Hungry for Change
Added Rotten
Added Cooked
Added Forks Over Knives
Added The Magic Pill


In [475]:
set(test.param)

{'cooking',
 'diabetes',
 'energy',
 'fats',
 'general',
 'meat',
 'natural',
 'pesticides'}

In [453]:
movie_search('Jiro Dreams of Sushi', test_cats)

{'Michelin', 'JIRO', 'DREAMS', 'located', 'appearances', 'greatest', 'bar', 'world', 'and', 'Jiro', 'rating', 'a', 'kind', 'SUSHI', 'subway', 'story', 'from', '85', 'OF', 'three', 'sushi', 'at', 'out', 'first', 'lovers', 'calling', 'is', 'prestigious', 'Ono', 'seat', 'awarded', 'top', 'coveted', 'Tokyo', 'Sukiyabashi', '10', 'for', 'station', 'repeated', 's', 'by', 'year', 'the', 'proprietor', 'around', 'dollar', 'inauspiciously', 'chef', 'be', 'Despite', 'considered', 'advance', 'to', 'in', 'its', 'pilgrimage', 'months', 'He', 'of', 'humble', 'many', 'star', 'restaurant', 'make', 'shelling', 'only', 'it', 'globe', 'Guide', 'old'}


,source,image_url,artworkUrl30,artworkUrl60,collectionExplicitness,collectionHdPrice,collectionPrice,contentAdvisoryRating,country,currency,kind,description,previewUrl,primaryGenreName,date,trackCensoredName,trackExplicitness,trackHdPrice,trackHdRentalPrice,trackId,title,trackPrice,trackRentalPrice,length,web_url,wrapperType,source_id,formality,medium,param
0,David Gelb,https://is5-ssl.mzstatic.com/image/thumb/Video128/v4/a7/6c/23/a76c2355-b63b-ffeb-1080-5adeabd78fc8/source/100x100bb.jpg,https://is5-ssl.mzstatic.com/image/thumb/Video128/v4/a7/6c/23/a76c2355-b63b-ffeb-1080-5adeabd78fc8/source/30x30bb.jpg,https://is5-ssl.mzstatic.com/image/thumb/Video128/v4/a7/6c/23/a76c2355-b63b-ffeb-1080-5adeabd78fc8/source/60x60bb.jpg,notExplicit,12.99,9.99,PG,USA,USD,feature-movie,"JIRO DREAMS OF SUSHI is the story of 85-year-old Jiro Ono, considered by many to be the world’s greatest sushi chef. He is the proprietor of Sukiyabashi Jiro, a 10-seat, sushi-only restaurant inauspiciously located in a Tokyo subway station. Despite its humble appearances, it is the first restaurant of its kind to be awarded a prestigious three-star Michelin Guide rating, and sushi lovers from around the globe make repeated pilgrimage, calling months in advance and shelling out top dollar for a coveted seat at Jiro’s sushi bar.",https://video-ssl.itunes.apple.com/apple-assets-us-std-000001/Video127/v4/19/ed/7a/19ed7a3b-d109-6050-1c74-8a4cb7fc058e/mzvf_7123981860068183185.640x478.h264lc.U.p.m4v,Documentary,2012-03-09,Jiro Dreams of Sushi,notExplicit,12.99,3.99,542088376,Jiro Dreams of Sushi,9.99,3.99,82.0,https://itunes.apple.com/us/movie/jiro-dreams-of-sushi/id542088376?uo=4,track,Itunes Movie,Formal,video,sushi
